In [1]:
%env SPARK_LOCAL_HOSTNAME=localhost
from pyspark import SparkContext
from pyspark import SparkConf
from pyspark.sql import SparkSession
#spark configuration
conf = (
    SparkConf()
        .setMaster('local[*]')
        #.set('spark.driver.memory', '16g')
)

sc = SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()



env: SPARK_LOCAL_HOSTNAME=localhost


In [2]:
from tick_util import summarize_ticks

In [3]:
import re
# take in the file path, and return the ticker and type e.g. ('EQNR,TRADES') or ('NHY,BIDASK')
def split_file_path(file_path):
    file_name = file_path.split('/')[-1]
    file_name = file_name.split('.csv')[0]
    fields = file_name.split('_')
    return fields[0] + ',' + fields[2]

def get_key(line, ts_offset=13):
        # a very limited number of trades will have TickAttribLast variable set, this field JSON-like surrounded by quotes & w/ internal commas
        if '"' in line:
            tick_line = re.split(r',(?=")', line)
        else:
            tick_line = line.split(',')
        ticker = tick_line[0]
        key = ticker #+ '_' + tick_line[2][:ts_offset] # timestamp is 1st element of tick_line list, slice this string according to agg. period
        return key, line

files_rdd_in = sc.wholeTextFiles('data\SALME*', 2000) \
    .filter(lambda l: l != '' and l is not None) \
    .flatMap(lambda f :[split_file_path(f[0]) + "," + line for line in f[1].splitlines()[1:]]) \
    .map(get_key)


In [4]:
files_rdd_in.take(5)

[('SALME',
  'SALME,BIDASK,2021-08-16 08:59:58+02:00,1629097198,,8.0,7.71,8230,873'),
 ('SALME',
  'SALME,BIDASK,2021-08-16 09:00:14+02:00,1629097214,,7.86,7.88,5089,19702'),
 ('SALME',
  'SALME,BIDASK,2021-08-16 09:00:49+02:00,1629097249,,7.85,7.88,22,19702'),
 ('SALME',
  'SALME,BIDASK,2021-08-16 09:00:49+02:00,1629097249,,7.85,7.88,22,21382'),
 ('SALME',
  'SALME,BIDASK,2021-08-16 09:00:49+02:00,1629097249,,7.85,7.88,22,19702')]

In [6]:
files_rdd_in.count()

591094

In [ ]:
files_rdd_reduced = files_rdd_in.reduceByKey(lambda a, b: a + b)

In [ ]:
files_rdd_reduced.take(1)